# 2. CNN을 이용한 MNIST Dataset 학습 및 테스트 실습  
  
  
이번 시간에는 MNIST Dataset을 간단한 Convolutional Neural Network를 이용하여 학습시키고 훈련된 모델을 이용하여 0~9까지의 숫자를 Classification하는 실습을 해보겠습니다.

In [ ]:
## 필요한 라이브러리 임포트
import torch
import numpy as np

**MNIST 데이터 로드**  
저번 시간에 이미 다운로드를 했기 때문에, 다운로드 옵션을 False로 변경하고 데이터 로더를 준비합니다.

In [ ]:
from torchvision import datasets as dsets
import torchvision.transforms as transforms

# 데이터로드에 사용할 서브 프로세스 수
num_workers = 0
# 배치당 로드할 샘플 수
batch_size = 20

# 데이터를 Float Tensor형으로 변환합니다.
transform = transforms.ToTensor()

# 학습 및 테스트 데이터 로드
train_data = dsets.MNIST(root='data', train=True, download=False, transform=transform)
test_data = dsets.MNIST(root='data', train=False, download=False, transform=transform)

# 학습 및 테스트 데이터 로더 준비
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

**간단한 CNN 설계하기**  
저번 시간에 이미 MNIST 데이터를 시각화 해보고 어떠한 데이터셋인지 인지하였기에 이번 시간에 그 과정은 생략하고, CNN 구조를 설계해보도록 하겠습니다.  
  
2개의 Convolution layer와 2개의 Pooling Layer, 3개의 Fully-connected layer를 가진 간단한 CNN을 설계해보겠습니다.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        ## nn.Conv2d(입력 채널 수, 출력 채널 수, 커널 크기, 스트라이드)
        conv1 = nn.Conv2d(1, 6, 5, 1) # 출력 크기 = 24*24, 채널 수 = 6
        ## nn.MaxPool2d(커널 크기 및 스트라이드)
        pool1 = nn.MaxPool2d(2) # 출력 크기 = 12*12
        conv2 = nn.Conv2d(6, 16, 5, 1) # 출력 크기 = 8*8, 채널 수 = 16
        pool2 = nn.MaxPool2d(2) # 출력 크기 = 4*4
        
        self.conv_module = nn.Sequential(
            conv1,
            nn.ReLU(),
            pool1,
            conv2,
            nn.ReLU(),
            pool2
        )
        ## 채널 수 = 16, 출력 크기 = 4*4
        self.fc1 = nn.Linear(16*4*4, 10)
        
        
    def forward(self, x):
        x = self.conv_module(x)
        dim = 1
        for d in x.size()[1:]: #FC Layer의 입력 크기를 구한다
            dim = dim * d
        x = x.view(-1, dim)
        x = F.relu(self.fc1(x))
        return x

model = CNN()
print(model)

**Loss 함수 및 Optimizer 지정하기**
이번에는 Loss 함수는 CrossEntropyLoss를 사용하고, Optimizer는 Adam을 사용해보겠습니다.

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

**CNN 학습하기**  
위에서 설계한 CNN을 학습시켜 봅시다.
그래프를 통하여 Loss의 변화를 파악 하여 학습이 올바르게 이루어지고 있는지 확인할 수 있습니다.

In [ ]:
from livelossplot import PlotLosses

n_epochs = 20 # 학습 epoch 지정
liveloss = PlotLosses()
model.train() 

for epoch in range(n_epochs):
    logs = {}
    train_loss = 0.0
    ###################
    #    모델 학습    #
    ###################
    for data, target in train_loader:
        # 모든 optimizer 변수와 gradients를 초기화
        optimizer.zero_grad()
        # 정방향 학습 : 입력을 모델로 전달하여 예측된 출력 계산
        output = model(data)
        # Loss 계산
        loss = criterion(output, target)
        # 역전파 : 모델의 매개변수를 고려하여 loss의 gradients를 계산
        loss.backward()
        # 매개변수 업데이트
        optimizer.step()
        # 훈련 Loss 업데이트
        train_loss += loss.item()*data.size(0)
        
        visualize_loss = train_loss/len(train_loader.dataset)
        logs['train_loss'] = visualize_loss
    liveloss.update(logs)
    liveloss.draw()
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        visualize_loss
        ))

**훈련된 모델 테스트 해보기**

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()

for data, target in test_loader:
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

**테스트 결과를 시각화 하기**  
각 숫자당 모델이 예측한 값과 실제 값을 시각화합니다.

In [ ]:
import matplotlib.pyplot as plt
dataiter = iter(test_loader)
images, labels = dataiter.next()

output = model(images)
_, preds = torch.max(output, 1)
images = images.numpy()

fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title("{} ({})".format(str(preds[idx].item()), str(labels[idx].item())),
                 color=("green" if preds[idx]==labels[idx] else "red"))